<h2>Direcciones de ruta</h2>

Para proceder, en primer lugar, <b>establecemos las rutas necesarias</b>. Estas serán las rutas de las carpetas y archivos que utilizaremos.
Es <b>recomendable ejecutar la siguiente celda al comienzo de cualquier sesión</b>, ya que estas constantes se usan en diferentes procesos (extracción de features, análisis etc):

In [ ]:
# DIRECCIONES: Introducir direcciones de ruta de los videos, del ejecutable de
# openSmile, y de la configuración de openSmile. Indicar también dónde se desean
# almacenar los audios y los .csv que se extraen.

videos_path = ''
opensmile_path = ''
config_path = ''
audios_path = ''
audios_cut_path = ''
csv_path = ''

<h2>Extracción de Features</h2>

El siguiente bloque de código incluye las funciones de Python que se van a utilizar para realizar el procesado de los audios, desde la extracción de audio a partir de los vídeos hasta la extracción de las propias features.

Antes de nada, <b>OpenSmile debe haber sido descargado e instalado</b>, preferiblemente en una carpeta 'OpenSmile' que se encuentre en la misma carpeta que el proyecto de Python, para <b>facilitar el uso de rutas relativas</b>. Ver Paso 1 en las Instrucciones o la [guía de OpenSmile](https://audeering.github.io/opensmile/get-started.html#obtaining-and-installing-opensmile).

In [ ]:
from moviepy.editor import VideoFileClip
import os
import subprocess
from pydub import AudioSegment


class FeatureExtractor:
    def __init__(self, ruta_videos, ruta_audios, ruta_opensmile, ruta_config, ruta_csv, ruta_audios_cut):
        self.videos_path = ruta_videos
        self.opensmile_path = ruta_opensmile
        self.config_path = ruta_config
        self.audios_path = ruta_audios
        self.csv_path = ruta_csv
        self.audios_cut_path = ruta_audios_cut

    def extraer_audio(self):
        for video in os.listdir(self.videos_path):
            if video.endswith(".MOV") or video.endswith(".mp4") or video.endswith(".MTS"):
                video_py = VideoFileClip(f'{self.videos_path}/{video}')
                audio_py = video_py.audio
                audio_py.write_audiofile(f'{self.audios_path}/{os.path.splitext(video)[0]}.wav', codec='pcm_s16le',
                                         ffmpeg_params=["-ac", "1"])

    def cortar_audio(self, start, end):
        for audio in os.listdir(self.audios_path):
            if audio.endswith(".wav"):
                start_ms = start * 1000  # Where the first number is the second. The same happens with end_ms
                end_ms = end * 1000
                audio_path = AudioSegment.from_file(f'{self.audios_path}/{os.path.splitext(audio)[0]}.wav')
                output_audio = audio_path[start_ms:end_ms]
                output_audio.export(f'{self.audios_cut_path}/{os.path.splitext(audio)[0]}.wav', format="wav")

    def extraer_features(self):
        if not os.listdir(self.audios_cut_path):
            self.audios_cut_path = self.audios_path

        for audio in os.listdir(self.audios_cut_path):

            if audio.endswith(".wav"):
                comando = (
                    f'"{self.opensmile_path}" '
                    f'-C "{self.config_path}" '
                    f'-I "{os.path.join(self.audios_cut_path, audio)}" '
                    f'-O "{os.path.join(self.csv_path, os.path.splitext(audio)[0])}.csv"'
                )
                subprocess.run(comando, shell=True)


Antes de procededer, debemos configurar el número de frames y las porciones de audio a analizar deben modificarse manualmente en el archivo:

<b>'config/shared/FrameModeFunctionals.conf'</b>

que se encontrará en la carpeta principal de OpenSmile. Ver Paso 2 en el documento 'Instrucciones', o la [guía de openSmile](https://audeering.github.io/opensmile/get-started.html#common-options-for-all-standard-configuration-files).

A continuación, llamamos a la clase que hemos creado anteriormente. Ahora podremos ejecutar cada una de las funciones que deseemos.

In [ ]:
audio_processor = FeatureExtractor(videos_path, audios_path, opensmile_path, config_path, csv_path, audios_cut_path)

<b>Extracción de audios desde los vídeos</b>. Los vídeos deberían estr en formato MOV o mp4, y los audios se extraerán en formato wav.

In [ ]:
audio_processor.extraer_audio()

<b>Cortar los audios para que tengan una duración homogénea y en los mismos segmentos</b>. 'start' indica a partir de qué segundo queremos cortar, y 'end' en qué segundo queremos finalizar.

In [ ]:
audio_processor.cortar_audio(start = 10, end = 70)

<b>Extracción de las features</b>. Se seguirá la configuración especificada en la constante 'config_path' y el FrameMode en 'config/shared/FrameModeFunctionals.conf':

In [ ]:
audio_processor.extraer_features()